In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [62]:
Folder_A = 'A.The_full_year_MILP'
Folder_B = 'B.Operation_cost'
Folder_D = 'D.Representative_days_based_on_RES_and_Demand'
Folder_E = 'E.Representative_days_based_on_Line_Benefits_OptModel'
Folder_K = 'K.Investments_per_hour'
Folder_L = 'L.Cont_Investments_per_hour'

category_dict = { "FYMILP": Folder_A,
                 "OPC":Folder_B,
                 "R&D": Folder_D ,
                 "OPT_LB": Folder_E,
                 "HI": Folder_K,
                  "CHI": Folder_L,
                }

## Test new way 

In [63]:
CaseName_Base = "9n_mod1"
cm = "CHI"
nbc = 10
folder_name = category_dict[cm]


In [12]:
destination_folder = f"../Y.FYTS_from_ByStages_2/{CaseName_Base}/{cm}"
filename = f"NetworkUtilization_nc{nbc}.csv"

# # Read input data
# df_duration = pd.read_csv(
#     f"../{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/2.Par/oT_Data_Duration_{CaseName_Base}_ByStages_nc{nbc}.csv")
# df_ts_bs = pd.read_csv(
#     f"../{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/4.OutWoInv/oT_Result_NetworkUtilizationPerNode_DC_{CaseName_Base}_ByStages_nc{nbc}.csv")

# Read input data
df_duration = pd.read_csv(
    f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/2.Par/oT_Data_Duration_{CaseName_Base}_ByStages_nc{nbc}.csv")
df_ts_bs = pd.read_csv(
    f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/4.OutWoInv/oT_Result_NetworkUtilizationPerNode_DC_{CaseName_Base}_ByStages_nc{nbc}.csv")


# Ensure data types are consistent
df_duration["LoadLevel"] = df_duration["LoadLevel"].astype(str)
df_ts_bs["LoadLevel"] = df_ts_bs["LoadLevel"].astype(str)

# Create mapping of load level to stage name
load_level_stage_map = df_duration.set_index("LoadLevel")["Stage"].to_dict()

# Vectorize the mapping instead of looping
df_duration['MappedStage'] = df_duration['LoadLevel'].map(load_level_stage_map)
df_ts_bs['MappedStage'] = df_ts_bs['LoadLevel'].map(load_level_stage_map)

# Filter out line candidates
df_ts_bs = df_ts_bs[df_ts_bs["Circuit"] == "eac1"]

# Merge df_duration with df_ts_bs using relevant keys
merged_df = df_ts_bs.merge(df_duration, on=['LoadLevel'], how='inner')

# Pivoting instead of loops
pivot_df = merged_df.pivot_table(index='LoadLevel',
                                 columns=['InitialNode', 'FinalNode'],
                                 values='GWh', aggfunc='first').fillna(0)


In [20]:
df_duration

,LoadLevel,Duration,Stage,MappedStage
0,01-01 00:00:00+01:00,0,sti6,sti6
1,01-01 01:00:00+01:00,0,sti6,sti6
2,01-01 02:00:00+01:00,0,sti6,sti6
3,01-01 03:00:00+01:00,0,sti6,sti6
4,01-01 04:00:00+01:00,0,sti6,sti6
...,...,...,...,...
8731,12-30 19:00:00+01:00,0,sti1,sti1
8732,12-30 20:00:00+01:00,0,sti1,sti1
8733,12-30 21:00:00+01:00,0,sti1,sti1
8734,12-30 22:00:00+01:00,0,sti1,sti1


In [16]:
load_level_stage_map

{'01-01 00:00:00+01:00': 'sti6',
 '01-01 01:00:00+01:00': 'sti6',
 '01-01 02:00:00+01:00': 'sti6',
 '01-01 03:00:00+01:00': 'sti6',
 '01-01 04:00:00+01:00': 'sti6',
 '01-01 05:00:00+01:00': 'sti6',
 '01-01 06:00:00+01:00': 'sti6',
 '01-01 07:00:00+01:00': 'sti6',
 '01-01 08:00:00+01:00': 'sti6',
 '01-01 09:00:00+01:00': 'sti6',
 '01-01 10:00:00+01:00': 'sti6',
 '01-01 11:00:00+01:00': 'sti6',
 '01-01 12:00:00+01:00': 'sti6',
 '01-01 13:00:00+01:00': 'sti6',
 '01-01 14:00:00+01:00': 'sti6',
 '01-01 15:00:00+01:00': 'sti6',
 '01-01 16:00:00+01:00': 'sti6',
 '01-01 17:00:00+01:00': 'sti6',
 '01-01 18:00:00+01:00': 'sti6',
 '01-01 19:00:00+01:00': 'sti6',
 '01-01 20:00:00+01:00': 'sti6',
 '01-01 21:00:00+01:00': 'sti6',
 '01-01 22:00:00+01:00': 'sti6',
 '01-01 23:00:00+01:00': 'sti6',
 '01-02 00:00:00+01:00': 'sti6',
 '01-02 01:00:00+01:00': 'sti6',
 '01-02 02:00:00+01:00': 'sti6',
 '01-02 03:00:00+01:00': 'sti6',
 '01-02 04:00:00+01:00': 'sti6',
 '01-02 05:00:00+01:00': 'sti6',
 '01-02 06

In [15]:
df_ts_bs

,Period,Scenario,LoadLevel,InitialNode,FinalNode,Circuit,GWh,MappedStage
0,2030,sc01,01-07 16:00:00+01:00,Node_2,Node_3,eac1,0.358197,sti5
3,2030,sc01,01-07 16:00:00+01:00,Node_6,Node_7,eac1,0.513296,sti5
10,2030,sc01,01-07 16:00:00+01:00,Node_8,Node_9,eac1,0.431873,sti5
13,2030,sc01,01-07 16:00:00+01:00,Node_3,Node_4,eac1,0.999997,sti5
20,2030,sc01,01-07 16:00:00+01:00,Node_7,Node_8,eac1,0.045012,sti5
...,...,...,...,...,...,...,...,...
459,2030,sc01,09-25 20:00:00+02:00,Node_2,Node_6,eac1,0.255013,sti1
466,2030,sc01,09-25 20:00:00+02:00,Node_4,Node_9,eac1,0.841467,sti1
469,2030,sc01,09-25 20:00:00+02:00,Node_4,Node_6,eac1,0.475164,sti1
475,2030,sc01,09-25 20:00:00+02:00,Node_6,Node_8,eac1,0.384127,sti1


In [13]:
pivot_df

InitialNode             Node_1    Node_2              Node_3             
FinalNode               Node_6    Node_3    Node_6    Node_4    Node_6   
LoadLevel                                                                
01-07 16:00:00+01:00  0.999997  0.358197  0.143123  0.999997  0.274514  \
01-10 09:00:00+01:00  0.999997  0.827952  0.082703  0.999997  0.469111   
01-14 20:00:00+01:00  0.999997  0.030997  0.311895  0.708009  0.223429   
01-29 10:00:00+01:00  0.999997  0.999997  0.207764  0.999997  0.361489   
01-29 11:00:00+01:00  0.999997  0.999997  0.330395  0.999997  0.279735   
01-30 08:00:00+01:00  0.999997  0.359957  0.382678  0.999997  0.435097   
04-13 11:00:00+02:00  0.994157  0.028028  0.680289  0.423638  0.467540   
07-01 17:00:00+02:00  0.999997  0.461285  0.305359  0.999997  0.027069   
08-19 12:00:00+02:00  0.676033  0.228888  0.999997  0.363039  0.552220   
09-25 20:00:00+02:00  0.267383  0.110796  0.255013  0.489775  0.225407   

InitialNode             Node_4                        Node_6             
FinalNode               Node_5    Node_6    Node_9    Node_7    Node_8   
LoadLevel                                                                
01-07 16:00:00+01:00  0.999997  0.419112  0.120350  0.513296  0.579946  \
01-10 09:00:00+01:00  0.999997  0.273165  0.123581  0.434312  0.454252   
01-14 20:00:00+01:00  0.999997  0.610077  0.838228  0.415512  0.223788   
01-29 10:00:00+01:00  0.999997  0.353881  0.257529  0.262849  0.214271   
01-29 11:00:00+01:00  0.999997  0.415197  0.396072  0.163298  0.134884   
01-30 08:00:00+01:00  0.999997  0.298675  0.148418  0.484613  0.503097   
04-13 11:00:00+02:00  0.999997  0.615429  0.589737  0.004979  0.079897   
07-01 17:00:00+02:00  0.999997  0.604696  0.225938  0.151940  0.349608   
08-19 12:00:00+02:00  0.999997  0.641064  0.929070  0.390140  0.246825   
09-25 20:00:00+02:00  0.999997  0.475164  0.841467  0.570467  0.384127   

InitialNode             Node_7    Node_8  
FinalNode               Node_8    Node_9  
LoadLevel                                 
01-07 16:00:00+01:00  0.045012  0.431873  
01-10 09:00:00+01:00  0.083652  0.259708  
01-14 20:00:00+01:00  0.343533  0.043987  
01-29 10:00:00+01:00  0.126434  0.014336  
01-29 11:00:00+01:00  0.076342  0.041783  
01-30 08:00:00+01:00  0.098048  0.304280  
04-13 11:00:00+02:00  0.107339  0.131929  
07-01 17:00:00+02:00  0.209099  0.502656  
08-19 12:00:00+02:00  0.276679  0.046951  
09-25 20:00:00+02:00  0.375542  0.056106

In [64]:
destination_folder = f"../Y.FYTS_from_ByStages/{CaseName_Base}/{cm}"

filename = f"NetworkUtilization_nc{nbc}.csv"

# Read input data
df_duration = pd.read_csv(
    f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/2.Par/oT_Data_Duration_{CaseName_Base}_ByStages_nc{nbc}.csv")
df_ts_bs = pd.read_csv(
    f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/4.OutWoInv/oT_Result_NetworkUtilizationPerNode_DC_{CaseName_Base}_ByStages_nc{nbc}.csv")

# Ensure data types are consistent: convert load levels to strings
df_duration["LoadLevel"] = df_duration["LoadLevel"].astype(str)
df_ts_bs["LoadLevel"] = df_ts_bs["LoadLevel"].astype(str)
df_ts_bs["InitialNode"] = df_ts_bs["InitialNode"].astype(str)
df_ts_bs["FinalNode"] = df_ts_bs["FinalNode"].astype(str)

# Filter out line candidates
df_ts_bs = df_ts_bs[df_ts_bs["Circuit"] == "eac1"]

# Create mapping of load level to stage name
load_level_to_stage_map = df_duration.set_index("LoadLevel")["Stage"].to_dict()
all_load_levels = df_duration.LoadLevel

# Create dataframe that that wil hold all timeseries
frame_values = pd.DataFrame({"LoadLevel": all_load_levels})

# Get unique node pairs connected by an existing line
unique_node_pairs = [tuple(x) for x in df_ts_bs[['InitialNode', 'FinalNode']].drop_duplicates().values.tolist()]

# Create a MultiIndex from the unique node pairs
multi_index = pd.MultiIndex.from_tuples(unique_node_pairs, names=['InitialNode', 'FinalNode'])

# Initialize a DataFrame to hold all time series with MultiIndex
frame_values = pd.DataFrame(index=all_load_levels, columns=multi_index)

node_pair = unique_node_pairs[0]

In [65]:
df_ts_bs

,Period,Scenario,LoadLevel,InitialNode,FinalNode,Circuit,GWh
0,2030,sc01,01-07 16:00:00+01:00,Node_2,Node_3,eac1,0.358197
3,2030,sc01,01-07 16:00:00+01:00,Node_6,Node_7,eac1,0.513296
10,2030,sc01,01-07 16:00:00+01:00,Node_8,Node_9,eac1,0.431873
13,2030,sc01,01-07 16:00:00+01:00,Node_3,Node_4,eac1,0.999997
20,2030,sc01,01-07 16:00:00+01:00,Node_7,Node_8,eac1,0.045012
...,...,...,...,...,...,...,...
459,2030,sc01,09-25 20:00:00+02:00,Node_2,Node_6,eac1,0.255013
466,2030,sc01,09-25 20:00:00+02:00,Node_4,Node_9,eac1,0.841467
469,2030,sc01,09-25 20:00:00+02:00,Node_4,Node_6,eac1,0.475164
475,2030,sc01,09-25 20:00:00+02:00,Node_6,Node_8,eac1,0.384127


In [59]:
stage_to_repr_load_level_map = df_duration[df_duration["Duration"] == 1].set_index("Stage")["LoadLevel"].to_dict()

for node_pair in unique_node_pairs[0:]:
    
    filter_node_pair = (df_ts_bs["InitialNode"] == node_pair[0]) & (df_ts_bs["FinalNode"] == node_pair[1])
    repr_load_level_to_value_map = df_ts_bs[filter_node_pair].set_index("LoadLevel")["GWh"].to_dict()
    
    frame_values[(node_pair[0], node_pair[1])] = frame_values.index.map(load_level_stage_map).map(stage_repr_load_level_map).map(repr_load_level_to_value_map)
    

    

In [60]:
frame_values

InitialNode             Node_2    Node_6    Node_8    Node_3    Node_7   
FinalNode               Node_3    Node_7    Node_9    Node_4    Node_8   
LoadLevel                                                                
01-01 00:00:00+01:00  0.028028  0.004979  0.131929  0.423638  0.107339  \
01-01 01:00:00+01:00  0.028028  0.004979  0.131929  0.423638  0.107339   
01-01 02:00:00+01:00  0.028028  0.004979  0.131929  0.423638  0.107339   
01-01 03:00:00+01:00  0.028028  0.004979  0.131929  0.423638  0.107339   
01-01 04:00:00+01:00  0.028028  0.004979  0.131929  0.423638  0.107339   
...                        ...       ...       ...       ...       ...   
12-30 19:00:00+01:00  0.110796  0.570467  0.056106  0.489775  0.375542   
12-30 20:00:00+01:00  0.110796  0.570467  0.056106  0.489775  0.375542   
12-30 21:00:00+01:00  0.110796  0.570467  0.056106  0.489775  0.375542   
12-30 22:00:00+01:00  0.110796  0.570467  0.056106  0.489775  0.375542   
12-30 23:00:00+01:00  0.110796  0.570467  0.056106  0.489775  0.375542   

InitialNode             Node_1    Node_4    Node_2    Node_4             
FinalNode               Node_6    Node_5    Node_6    Node_9    Node_6   
LoadLevel                                                                
01-01 00:00:00+01:00  0.994157  0.999997  0.680289  0.589737  0.615429  \
01-01 01:00:00+01:00  0.994157  0.999997  0.680289  0.589737  0.615429   
01-01 02:00:00+01:00  0.994157  0.999997  0.680289  0.589737  0.615429   
01-01 03:00:00+01:00  0.994157  0.999997  0.680289  0.589737  0.615429   
01-01 04:00:00+01:00  0.994157  0.999997  0.680289  0.589737  0.615429   
...                        ...       ...       ...       ...       ...   
12-30 19:00:00+01:00  0.267383  0.999997  0.255013  0.841467  0.475164   
12-30 20:00:00+01:00  0.267383  0.999997  0.255013  0.841467  0.475164   
12-30 21:00:00+01:00  0.267383  0.999997  0.255013  0.841467  0.475164   
12-30 22:00:00+01:00  0.267383  0.999997  0.255013  0.841467  0.475164   
12-30 23:00:00+01:00  0.267383  0.999997  0.255013  0.841467  0.475164   

InitialNode             Node_6    Node_3  
FinalNode               Node_8    Node_6  
LoadLevel                                 
01-01 00:00:00+01:00  0.079897  0.467540  
01-01 01:00:00+01:00  0.079897  0.467540  
01-01 02:00:00+01:00  0.079897  0.467540  
01-01 03:00:00+01:00  0.079897  0.467540  
01-01 04:00:00+01:00  0.079897  0.467540  
...                        ...       ...  
12-30 19:00:00+01:00  0.384127  0.225407  
12-30 20:00:00+01:00  0.384127  0.225407  
12-30 21:00:00+01:00  0.384127  0.225407  
12-30 22:00:00+01:00  0.384127  0.225407  
12-30 23:00:00+01:00  0.384127  0.225407  

[8736 rows x 12 columns]

In [49]:
filter_node_pair = (df_ts_bs["InitialNode"] == node_pair[0]) & (df_ts_bs["FinalNode"] == node_pair[1])
df_ts_bs[filter_node_pair]

,Period,Scenario,LoadLevel,InitialNode,FinalNode,Circuit,GWh
0,2030,sc01,01-07 16:00:00+01:00,Node_2,Node_3,eac1,0.358197
48,2030,sc01,01-10 09:00:00+01:00,Node_2,Node_3,eac1,0.827952
96,2030,sc01,01-14 20:00:00+01:00,Node_2,Node_3,eac1,0.030997
144,2030,sc01,01-29 10:00:00+01:00,Node_2,Node_3,eac1,0.999997
192,2030,sc01,01-29 11:00:00+01:00,Node_2,Node_3,eac1,0.999997
240,2030,sc01,01-30 08:00:00+01:00,Node_2,Node_3,eac1,0.359957
288,2030,sc01,04-13 11:00:00+02:00,Node_2,Node_3,eac1,0.028028
336,2030,sc01,07-01 17:00:00+02:00,Node_2,Node_3,eac1,0.461285
384,2030,sc01,08-19 12:00:00+02:00,Node_2,Node_3,eac1,0.228888
432,2030,sc01,09-25 20:00:00+02:00,Node_2,Node_3,eac1,0.110796


In [38]:
stage_to_repr_load_level_map = df_duration[df_duration["Duration"] == 1].set_index("Stage")["LoadLevel"].to_dict()

In [52]:
df_ts_bs[filter_node_pair].set_index("LoadLevel")["GWh"].to_dict()

{'01-07 16:00:00+01:00': 0.3581973491409712,
 '01-10 09:00:00+01:00': 0.8279516463882038,
 '01-14 20:00:00+01:00': 0.030996818013223,
 '01-29 10:00:00+01:00': 0.999996666677778,
 '01-29 11:00:00+01:00': 0.999996666677778,
 '01-30 08:00:00+01:00': 0.3599570827174022,
 '04-13 11:00:00+02:00': 0.0280283962201716,
 '07-01 17:00:00+02:00': 0.4612845458214514,
 '08-19 12:00:00+02:00': 0.2288884883717056,
 '09-25 20:00:00+02:00': 0.1107956219133337}

In [39]:
frame_values["LoadLevelMapped"] = frame_values.index.map(load_level_stage_map).map(stage_repr_load_level_map)

In [40]:
frame_values

InitialNode          Node_2 Node_6 Node_8 Node_3 Node_7 Node_1 Node_4 Node_2   
FinalNode            Node_3 Node_7 Node_9 Node_4 Node_8 Node_6 Node_5 Node_6   
LoadLevel                                                                      
01-01 00:00:00+01:00    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  \
01-01 01:00:00+01:00    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
01-01 02:00:00+01:00    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
01-01 03:00:00+01:00    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
01-01 04:00:00+01:00    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
...                     ...    ...    ...    ...    ...    ...    ...    ...   
12-30 19:00:00+01:00    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
12-30 20:00:00+01:00    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
12-30 21:00:00+01:00    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
12-30 22:00:00+01:00    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
12-30 23:00:00+01:00    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

InitialNode          Node_4        Node_6 Node_3       LoadLevelMapped  
FinalNode            Node_9 Node_6 Node_8 Node_6                        
LoadLevel                                                               
01-01 00:00:00+01:00    NaN    NaN    NaN    NaN  04-13 11:00:00+02:00  
01-01 01:00:00+01:00    NaN    NaN    NaN    NaN  04-13 11:00:00+02:00  
01-01 02:00:00+01:00    NaN    NaN    NaN    NaN  04-13 11:00:00+02:00  
01-01 03:00:00+01:00    NaN    NaN    NaN    NaN  04-13 11:00:00+02:00  
01-01 04:00:00+01:00    NaN    NaN    NaN    NaN  04-13 11:00:00+02:00  
...                     ...    ...    ...    ...                   ...  
12-30 19:00:00+01:00    NaN    NaN    NaN    NaN  09-25 20:00:00+02:00  
12-30 20:00:00+01:00    NaN    NaN    NaN    NaN  09-25 20:00:00+02:00  
12-30 21:00:00+01:00    NaN    NaN    NaN    NaN  09-25 20:00:00+02:00  
12-30 22:00:00+01:00    NaN    NaN    NaN    NaN  09-25 20:00:00+02:00  
12-30 23:00:00+01:00    NaN    NaN    NaN    NaN  09-25 20:00:00+02:00  

[8736 rows x 13 columns]

## Utilization per node Timeseries

In [7]:
def create_utilization_fyts_frame(CaseName_Base,cm,nbc):
    destination_folder = f"Y.FYTS_from_ByStages/{CaseName_Base}/{cm}"

    filename = f"NetworkUtilization_nc{nbc}.csv"


    # Read input data
    df_duration = pd.read_csv(f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/2.Par/oT_Data_Duration_{CaseName_Base}_ByStages_nc{nbc}.csv")
    df_ts_bs = pd.read_csv(f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/4.OutWoInv/oT_Result_NetworkUtilizationPerNode_DC_{CaseName_Base}_ByStages_nc{nbc}.csv")

    # Ensure data types are consistent: convert load levels to strings
    df_duration["LoadLevel"] = df_duration["LoadLevel"].astype(str)
    df_ts_bs["LoadLevel"] = df_ts_bs["LoadLevel"].astype(str)
    df_ts_bs["InitialNode"] = df_ts_bs["InitialNode"].astype(str)
    df_ts_bs["FinalNode"] = df_ts_bs["FinalNode"].astype(str)

    #Filter out line candidates
    df_ts_bs =df_ts_bs[df_ts_bs["Circuit"] == "eac1"]

    # Create mapping of load level to stage name
    load_level_stage_map = df_duration.set_index("LoadLevel")["Stage"].to_dict()
    all_load_levels = df_duration.LoadLevel

    #Create dataframe that that wil hold all timeseries
    frame_values = pd.DataFrame({"LoadLevel":all_load_levels})
    
    #Get unique node pairs connected by an existing line 
    unique_node_pairs = [tuple(x) for x in df_ts_bs[['InitialNode', 'FinalNode']].drop_duplicates().values.tolist()]

    # Create a MultiIndex from the unique node pairs
    multi_index = pd.MultiIndex.from_tuples(unique_node_pairs, names=['InitialNode', 'FinalNode'])

    # Initialize a DataFrame to hold all time series with MultiIndex
    frame_values = pd.DataFrame(index=all_load_levels, columns=multi_index)

    #Select one node pair
    node_pair = unique_node_pairs[0]

    for node_pair in unique_node_pairs:
        print(node_pair)

        initial_node, final_node = node_pair

        # Initialize the full-year time series array
        fy_ts = np.zeros(len(all_load_levels))
        for i, load_level in enumerate(all_load_levels):
            this_loadlevel_stage = load_level_stage_map[load_level]

            # Filter to find the correct reduced load level
            filtered_duration = df_duration[(df_duration["Stage"] == this_loadlevel_stage) & (df_duration["Duration"] == 1)]

            if not filtered_duration.empty:
                reduced_temp_load_level = filtered_duration.LoadLevel.iloc[0]

                # Find the corresponding value in df_ts_bs
                reduced_temp_value = df_ts_bs[(df_ts_bs["InitialNode"] == node_pair[0]) &(df_ts_bs["FinalNode"] == node_pair[1]) & (df_ts_bs["LoadLevel"] == str(reduced_temp_load_level))]

                if not reduced_temp_value.empty:
                    # Assuming you want to assign a value from reduced_temp_value to fy_ts
                    # You might need to aggregate if there are multiple values
                    fy_ts[i] = reduced_temp_value['GWh'].iloc[0]
        frame_values[(initial_node,final_node)] = fy_ts
    return frame_values,destination_folder,filename



In [9]:
#Define case 
CaseName_Base = 'RTS24_mod1'


cm = "CHI"
folder_name = category_dict[cm]

nbcs = [10,20,50,100,200]

for nbc in nbcs:
    frame_values,destination_folder,filename = create_utilization_fyts_frame(CaseName_Base,cm,nbc)
    # Check if the destination folder exists, if not, create it
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    #And write dataframe to destination
    frame_values.to_csv(os.path.join(destination_folder,filename))

('N_120', 'N_123')
('N_113', 'N_123')
('N_119', 'N_120')
('N_108', 'N_110')
('N_112', 'N_123')
('N_104', 'N_109')
('N_112', 'N_113')
('N_121', 'N_122')
('N_108', 'N_109')
('N_114', 'N_116')
('N_101', 'N_103')
('N_117', 'N_118')
('N_107', 'N_108')
('N_111', 'N_114')
('N_116', 'N_117')
('N_105', 'N_110')
('N_103', 'N_124')
('N_109', 'N_111')
('N_117', 'N_122')
('N_102', 'N_106')
('N_103', 'N_109')
('N_118', 'N_121')
('N_111', 'N_113')
('N_109', 'N_112')
('N_110', 'N_111')
('N_110', 'N_112')
('N_101', 'N_105')
('N_115', 'N_116')
('N_101', 'N_102')
('N_102', 'N_104')
('N_106', 'N_110')
('N_115', 'N_124')
('N_115', 'N_121')
('N_116', 'N_119')
('N_112', 'N_113')
('N_115', 'N_124')
('N_110', 'N_112')
('N_103', 'N_109')
('N_113', 'N_123')
('N_106', 'N_110')
('N_120', 'N_123')
('N_104', 'N_109')
('N_118', 'N_121')
('N_111', 'N_113')
('N_119', 'N_120')
('N_105', 'N_110')
('N_101', 'N_105')
('N_107', 'N_108')
('N_110', 'N_111')
('N_116', 'N_119')
('N_114', 'N_116')
('N_117', 'N_122')
('N_101', 'N

## Flow per node 

In [ ]:
def create_flow_fyts_frame(CaseName_Base,cm,nbc):
    destination_folder = f"Y.FYTS_from_ByStages/{CaseName_Base}/{cm}"

    filename = f"Flow_per_node_nc{nbc}.csv"


    # Read input data
    df_duration = pd.read_csv(f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/2.Par/oT_Data_Duration_{CaseName_Base}_ByStages_nc{nbc}.csv")
    df_ts_bs = pd.read_csv(f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/4.OutWoInv/oT_Result_NetworkFlowPerNode_{CaseName_Base}_ByStages_nc{nbc}.csv")

    # Ensure data types are consistent: convert load levels to strings
    df_duration["LoadLevel"] = df_duration["LoadLevel"].astype(str)
    df_ts_bs["LoadLevel"] = df_ts_bs["LoadLevel"].astype(str)
    df_ts_bs["InitialNode"] = df_ts_bs["InitialNode"].astype(str)
    df_ts_bs["FinalNode"] = df_ts_bs["FinalNode"].astype(str)

    #Filter out line candidates
    df_ts_bs =df_ts_bs[df_ts_bs["Circuit"] == "eac1"]

    # Create mapping of load level to stage name
    load_level_stage_map = df_duration.set_index("LoadLevel")["Stage"].to_dict()
    all_load_levels = df_duration.LoadLevel

    #Create dataframe that that wil hold all timeseries
    frame_values = pd.DataFrame({"LoadLevel":all_load_levels})
    
    #Get unique node pairs connected by an existing line 
    unique_node_pairs = [tuple(x) for x in df_ts_bs[['InitialNode', 'FinalNode']].drop_duplicates().values.tolist()]

    # Create a MultiIndex from the unique node pairs
    multi_index = pd.MultiIndex.from_tuples(unique_node_pairs, names=['InitialNode', 'FinalNode'])

    # Initialize a DataFrame to hold all time series with MultiIndex
    frame_values = pd.DataFrame(index=all_load_levels, columns=multi_index)

    #Select one node pair
    node_pair = unique_node_pairs[0]

    for node_pair in unique_node_pairs:
        print(node_pair)

        initial_node, final_node = node_pair

        # Initialize the full-year time series array
        fy_ts = np.zeros(len(all_load_levels))
        for i, load_level in enumerate(all_load_levels[:20]):
            this_loadlevel_stage = load_level_stage_map[load_level]

            # Filter to find the correct reduced load level
            filtered_duration = df_duration[(df_duration["Stage"] == this_loadlevel_stage) & (df_duration["Duration"] == 1)]

            if not filtered_duration.empty:
                reduced_temp_load_level = filtered_duration.LoadLevel.iloc[0]

                # Find the corresponding value in df_ts_bs
                reduced_temp_value = df_ts_bs[(df_ts_bs["InitialNode"] == node_pair[0]) &(df_ts_bs["FinalNode"] == node_pair[1]) & (df_ts_bs["LoadLevel"] == str(reduced_temp_load_level))]

                if not reduced_temp_value.empty:
                    # Assuming you want to assign a value from reduced_temp_value to fy_ts
                    # You might need to aggregate if there are multiple values
                    fy_ts[i] = reduced_temp_value['GWh'].iloc[0]
        frame_values[(initial_node,final_node)] = fy_ts
    return frame_values,destination_folder,filename



In [ ]:
#Define case 
CaseName_Base = 'RTS24_mod1'


cm = "CHI"
folder_name = category_dict[cm]

nbcs = [10,20,50,100,200]

for nbc in nbcs:
    frame_values,destination_folder,filename = create_flow_fyts_frame(CaseName_Base,cm,nbc)
    # Check if the destination folder exists, if not, create it
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    #And write dataframe to destination
    frame_values.to_csv(os.path.join(destination_folder,filename))

In [3]:
CaseName_Base = 'RTS24_mod1'

cm = "R&D"
folder_name = category_dict[cm]

nbc=20

destination_folder = f"Y.FYTS_from_ByStages/{CaseName_Base}/{cm}"

filename = f"NetworkUtilization_nc{nbc}.csv"


# Read input data
df_duration = pd.read_csv(f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/2.Par/oT_Data_Duration_{CaseName_Base}_ByStages_nc{nbc}.csv")
df_ts_bs = pd.read_csv(f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/4.OutWoInv/oT_Result_NetworkUtilizationPerNode_DC_{CaseName_Base}_ByStages_nc{nbc}.csv")

# Ensure data types are consistent: convert load levels to strings
df_duration["LoadLevel"] = df_duration["LoadLevel"].astype(str)
df_ts_bs["LoadLevel"] = df_ts_bs["LoadLevel"].astype(str)
df_ts_bs["InitialNode"] = df_ts_bs["InitialNode"].astype(str)
df_ts_bs["FinalNode"] = df_ts_bs["FinalNode"].astype(str)

#Filter out line candidates
#df_ts_bs =df_ts_bs[df_ts_bs["Circuit"] == "eac1"]

# Create mapping of load level to stage name
load_level_stage_map = df_duration.set_index("LoadLevel")["Stage"].to_dict()
all_load_levels = df_duration.LoadLevel

#Create dataframe that that wil hold all timeseries
frame_values = pd.DataFrame({"LoadLevel":all_load_levels})

# Select a unit
#units = np.unique(df_ts_bs.Unit)

# for unit in units:

#     # Initialize the full-year time series array
#     fy_ts = np.zeros(len(all_load_levels))

#     # Create full-year time series based on mapping and values of representative load levels
#     for i, load_level in enumerate(all_load_levels):
#         this_loadlevel_stage = load_level_stage_map[load_level]

#         # Filter to find the correct reduced load level
#         filtered_duration = df_duration[(df_duration["Stage"] == this_loadlevel_stage) & (df_duration["Duration"] == 1)]

#         if not filtered_duration.empty:
#             reduced_temp_load_level = filtered_duration.LoadLevel.iloc[0]

#             # Find the corresponding value in df_ts_bs
#             reduced_temp_value = df_ts_bs[(df_ts_bs["Unit"] == unit) & (df_ts_bs["LoadLevel"] == str(reduced_temp_load_level))]

#             if not reduced_temp_value.empty:
#                 # Assuming you want to assign a value from reduced_temp_value to fy_ts
#                 # You might need to aggregate if there are multiple values
#                 fy_ts[i] = reduced_temp_value['MW'].iloc[0]  # Replace 'YourValueColumn' with the actual column name
#     frame_values[unit] = fy_ts

In [ ]:
#Get unique node pairs connected by an existing line 
unique_node_pairs = [tuple(x) for x in df_ts_bs[['InitialNode', 'FinalNode']].drop_duplicates().values.tolist()]

# Create a MultiIndex from the unique node pairs
multi_index = pd.MultiIndex.from_tuples(unique_node_pairs, names=['InitialNode', 'FinalNode'])

# Initialize a DataFrame to hold all time series with MultiIndex
frame_values = pd.DataFrame(index=all_load_levels, columns=multi_index)

#Select one node pair
node_pair = unique_node_pairs[0]

for node_pair in unique_node_pairs:
    print(node_pair)

    initial_node, final_node = node_pair

    # Initialize the full-year time series array
    fy_ts = np.zeros(len(all_load_levels))
    for i, load_level in enumerate(all_load_levels):
        this_loadlevel_stage = load_level_stage_map[load_level]

        # Filter to find the correct reduced load level
        filtered_duration = df_duration[(df_duration["Stage"] == this_loadlevel_stage) & (df_duration["Duration"] == 1)]

        if not filtered_duration.empty:
            reduced_temp_load_level = filtered_duration.LoadLevel.iloc[0]

            # Find the corresponding value in df_ts_bs
            reduced_temp_value = df_ts_bs[(df_ts_bs["InitialNode"] == node_pair[0]) &(df_ts_bs["FinalNode"] == node_pair[1]) & (df_ts_bs["LoadLevel"] == str(reduced_temp_load_level))]

            if not reduced_temp_value.empty:
                # Assuming you want to assign a value from reduced_temp_value to fy_ts
                # You might need to aggregate if there are multiple values
                fy_ts[i] = reduced_temp_value['GWh'].iloc[0]  # Replace 'YourValueColumn' with the actual column name
    frame_values[(initial_node,final_node)] = fy_ts


('N_117', 'N_122')
('N_102', 'N_104')
('N_102', 'N_106')
('N_105', 'N_110')
('N_101', 'N_102')
('N_104', 'N_109')
('N_118', 'N_121')
('N_115', 'N_116')
('N_112', 'N_123')
('N_107', 'N_108')
('N_103', 'N_124')
('N_120', 'N_123')
('N_111', 'N_114')
('N_106', 'N_108')
('N_103', 'N_109')
('N_112', 'N_113')
('N_115', 'N_124')
('N_114', 'N_116')
('N_109', 'N_111')
('N_113', 'N_123')
('N_101', 'N_105')


In [15]:
frame_values

InitialNode              N_117     N_102               N_105    N_101   
FinalNode                N_122     N_104     N_106     N_110    N_102   
LoadLevel                                                               
01-01 00:00:00+01:00  0.002298  0.012218  0.276988  0.235467  0.66507  \
01-01 01:00:00+01:00  0.002298  0.012218  0.276988  0.235467  0.66507   
01-01 02:00:00+01:00  0.002298  0.012218  0.276988  0.235467  0.66507   
01-01 03:00:00+01:00  0.002298  0.012218  0.276988  0.235467  0.66507   
01-01 04:00:00+01:00  0.002298  0.012218  0.276988  0.235467  0.66507   
...                        ...       ...       ...       ...      ...   
12-30 19:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.00000   
12-30 20:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.00000   
12-30 21:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.00000   
12-30 22:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.00000   
12-30 23:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.00000   

InitialNode              N_104 N_118 N_115    N_112     N_107  ...     N_110   
FinalNode                N_109 N_121 N_116    N_123     N_108  ...     N_112   
LoadLevel                                                      ...             
01-01 00:00:00+01:00  0.487303   0.0   0.0  0.08008  0.178455  ...  0.109616  \
01-01 01:00:00+01:00  0.487303   0.0   0.0  0.08008  0.178455  ...  0.109616   
01-01 02:00:00+01:00  0.487303   0.0   0.0  0.08008  0.178455  ...  0.109616   
01-01 03:00:00+01:00  0.487303   0.0   0.0  0.08008  0.178455  ...  0.109616   
01-01 04:00:00+01:00  0.487303   0.0   0.0  0.08008  0.178455  ...  0.109616   
...                        ...   ...   ...      ...       ...  ...       ...   
12-30 19:00:00+01:00  0.000000   0.0   0.0  0.00000  0.000000  ...  0.000000   
12-30 20:00:00+01:00  0.000000   0.0   0.0  0.00000  0.000000  ...  0.000000   
12-30 21:00:00+01:00  0.000000   0.0   0.0  0.00000  0.000000  ...  0.000000   
12-30 22:00:00+01:00  0.000000   0.0   0.0  0.00000  0.000000  ...  0.000000   
12-30 23:00:00+01:00  0.000000   0.0   0.0  0.00000  0.000000  ...  0.000000   

InitialNode              N_119     N_121     N_101     N_106     N_108   
FinalNode                N_120     N_122     N_103     N_110     N_109   
LoadLevel                                                                
01-01 00:00:00+01:00  0.051525  0.002298  0.146579  0.079294  0.248013  \
01-01 01:00:00+01:00  0.051525  0.002298  0.146579  0.079294  0.248013   
01-01 02:00:00+01:00  0.051525  0.002298  0.146579  0.079294  0.248013   
01-01 03:00:00+01:00  0.051525  0.002298  0.146579  0.079294  0.248013   
01-01 04:00:00+01:00  0.051525  0.002298  0.146579  0.079294  0.248013   
...                        ...       ...       ...       ...       ...   
12-30 19:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.000000   
12-30 20:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.000000   
12-30 21:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.000000   
12-30 22:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.000000   
12-30 23:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.000000   

InitialNode                       N_110     N_116     N_115  
FinalNode               N_110     N_111     N_119     N_121  
LoadLevel                                                    
01-01 00:00:00+01:00  0.19354  0.076084  0.097851  0.581913  
01-01 01:00:00+01:00  0.19354  0.076084  0.097851  0.581913  
01-01 02:00:00+01:00  0.19354  0.076084  0.097851  0.581913  
01-01 03:00:00+01:00  0.19354  0.076084  0.097851  0.581913  
01-01 04:00:00+01:00  0.19354  0.076084  0.097851  0.581913  
...                       ...       ...       ...       ...  
12-30 19:00:00+01:00  0.00000  0.000000  0.000000  0.000000  
12-30 20:00:00+01:00  0.00000  0.000000  0.000000  0.000000  
12-30 21:00:00+01:00  0.00000  0.000000  0.000000  0.000000  
12-30 22:00:00+01:00  0.00000  0.000000  0.000000  0.000000  
12-30 23:00:0

In [6]:
frame_values

,LoadLevel,N_101-N_102
0,01-01 00:00:00+01:00,0.66507
1,01-01 01:00:00+01:00,0.66507
2,01-01 02:00:00+01:00,0.66507
3,01-01 03:00:00+01:00,0.66507
4,01-01 04:00:00+01:00,0.66507
...,...,...
8731,12-30 19:00:00+01:00,0.00000
8732,12-30 20:00:00+01:00,0.00000
8733,12-30 21:00:00+01:00,0.00000
8734,12-30 22:00:00+01:00,0.00000


In [ ]:
# Extract unique node pairs from df_ts_bs
unique_node_pairs = df_ts_bs[['InitialNode', 'FinalNode']].drop_duplicates().values

# Initialize a DataFrame to hold all time series
frame_values = pd.DataFrame({"LoadLevel": all_load_levels})

# Iterate over each unique node pair
for node_pair in unique_node_pairs:
    initial_node, final_node = node_pair

    # Initialize the full-year time series array for the current node pair
    fy_ts = np.zeros(len(all_load_levels))
    
    for i, load_level in enumerate(all_load_levels):
        this_loadlevel_stage = load_level_stage_map[load_level]

        # Filter to find the correct reduced load level
        filtered_duration = df_duration[
            (df_duration["Stage"] == this_loadlevel_stage) & 
            (df_duration["Duration"] == 1)
        ]

        if not filtered_duration.empty:
            reduced_temp_load_level = filtered_duration.LoadLevel.iloc[0]

            # Find the corresponding value in df_ts_bs
            reduced_temp_value = df_ts_bs[
                (df_ts_bs["InitialNode"] == initial_node) &
                (df_ts_bs["FinalNode"] == final_node) &
                (df_ts_bs["LoadLevel"] == str(reduced_temp_load_level))
            ]

            if not reduced_temp_value.empty:
                # Assign the value to the time series array
                fy_ts[i] = reduced_temp_value['GWh'].iloc[0]

    # Add the time series for the current node pair to the DataFrame
    frame_values[f"{initial_node}-{final_node}"] = fy_ts

# Print the first node pair's time series as an example
first_node_pair = unique_node_pairs[0]
first_node_pair_key = f"{first_node_pair[0]}-{first_node_pair[1]}"
print(f"Time series for node pair {first_node_pair_key}:")
print(frame_values[first_node_pair_key])


In [47]:
fy_ts[15]

0.0740160494921053

In [13]:
load_level = all_load_levels[0]
load_level

'01-01 00:00:00+01:00'

In [7]:
df_ts_bs.GWh.sum()

82.92351985661828

In [8]:
df_ts_bs[df_ts_bs["Circuit"] == "eac1"].GWh.sum()

82.92351985661828